<h1><center>The Battle of the Neighborhoods - Boston</center></h1>

<h1><center>Applied Data Science Capstone Project</center></h1>

<h4>About this Project</h4>
In this project, we will evaluate the Boston area neighborhoods in order to determine the most advantagous areas to live based on a customer's preferences.  

<h4>Discription and background of the problem</h4>
A potential job opening is available in the greater Boston area and a family is considering the best location to search for future homes.  The family has never been to Boston and unfamiliar with the various neighborhoods.  The family has an opportunity to house hunt during a four-day weekend and would like to search neighborhoods that offer the best opportunities as places with live with services and businesses that they frequent.  

<h4>Data and how it will solve the problem</h4>
The Foursquare.com data provides location information on different types of venues by category and name.  The family would like to search neighborhoods based on the proximity to favorite venues.

The family has a number of store preferences and would like to exam the types of venues in Foursquare.com data.  They would like to search specific names of businesses and determine if they can identify the locations with the best range to their preferred venues.

In [1]:
#Prepare the environment
import numpy as np 
import pandas as pd

In [2]:
#Preparing the dependencies
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip install lxml

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
import requests # library to handle requests
import random # library for random number generation
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [4]:
CLIENT_ID = 'PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH' # your Foursquare ID
CLIENT_SECRET = 'RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH
CLIENT_SECRET:RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE


In [5]:
address = '201 Edgewater Drive, Wakefield, MA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.5199907 -71.04339433612998


In [6]:
search_query = 'store'
radius = 10000
print(search_query + ' .... OK!')

store .... OK!


In [7]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.5199907,-71.04339433612998&v=20180604&query=store&radius=10000&limit=1000'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e4ab640a7ea001c5bbd25'},
 'response': {'venues': [{'id': '5be1033e92e7a9002c62ce84',
    'name': 'TUMI Store - Lynnfield',
    'location': {'address': '600 Market St Spc 687',
     'lat': 42.5164879,
     'lng': -71.0353819,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.5164879,
       'lng': -71.0353819}],
     'distance': 764,
     'postalCode': '01940',
     'cc': 'US',
     'city': 'Lynnfield',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['600 Market St Spc 687',
      'Lynnfield, MA 01940',
      'United States']},
    'categories': [{'id': '52f2ab2ebcbc57f1066b8b29',
      'name': 'Luggage Store',
      'pluralName': 'Luggage Stores',
      'shortName': 'Luggage Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580092264',
    'hasPerk': False},
   {'id': '57e39d8f498ee24a37e658d

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52f2ab2ebcbc57f1066b8b29', 'name': 'L...",False,5be1033e92e7a9002c62ce84,600 Market St Spc 687,US,Lynnfield,United States,NaN,764,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,TUMI Store - Lynnfield,v-1580092264,NaN
1,"[{'id': '5453de49498eade8af355881', 'name': 'B...",False,57e39d8f498ee24a37e658dc,601 Edgewater Dr,US,Wakefield,United States,NaN,554,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,XFINITY Store by Comcast,v-1580092264,NaN
2,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",False,5241ecb111d2f9f64c050071,NaN,US,Lynnfield,United States,NaN,1089,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.51442, 'lng': ...",42.514420,-71.032470,NaN,MA,The Paper Store,v-1580092264,NaN
3,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",False,5e0a669353f5420008d40710,225 Market Street,US,Lynnfield,United States,NaN,1206,"[225 Market Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5151073, 'lng'...",42.515107,-71.030266,01940,MA,AT&T Store,v-1580092264,NaN
4,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,50dc9fc1e4b067e5d7dbfafe,NaN,US,Wakefield,United States,NaN,1690,"[Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.517091, 'lng':...",42.517091,-71.063617,01880,MA,Medicine Store,v-1580092264,NaN


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,TUMI Store - Lynnfield,Luggage Store,600 Market St Spc 687,US,Lynnfield,United States,NaN,764,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,5be1033e92e7a9002c62ce84
1,XFINITY Store by Comcast,Business Service,601 Edgewater Dr,US,Wakefield,United States,NaN,554,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,57e39d8f498ee24a37e658dc
2,The Paper Store,Gift Shop,NaN,US,Lynnfield,United States,NaN,1089,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.51442, 'lng': ...",42.514420,-71.032470,NaN,MA,5241ecb111d2f9f64c050071
3,AT&T Store,Mobile Phone Shop,225 Market Street,US,Lynnfield,United States,NaN,1206,"[225 Market Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5151073, 'lng'...",42.515107,-71.030266,01940,MA,5e0a669353f5420008d40710
4,Medicine Store,Pharmacy,NaN,US,Wakefield,United States,NaN,1690,"[Wakefield, MA 01880, United States]","[{'label': 'display', 'lat': 42.517091, 'lng':...",42.517091,-71.063617,01880,MA,50dc9fc1e4b067e5d7dbfafe
5,C Store,Food & Drink Shop,41 Main St,US,Stoneham,United States,North Street,5282,"[41 Main St (North Street), Stoneham, MA, Unit...","[{'label': 'display', 'lat': 42.499152, 'lng':...",42.499152,-71.101230,NaN,MA,4cdb03cd99026dcb52060a81
6,"Your CBD Store - Reading, MA",Alternative Healer,587 Main St.,US,Reading,United States,NaN,4888,"[587 Main St., Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.5233339, 'lng'...",42.523334,-71.102806,01867,MA,5c639ab6838e5900396bbedd
7,XFINITY Store by Comcast,Business Service,8 Center Vil,US,Lynnfield,United States,NaN,1511,"[8 Center Vil, Lynnfield, MA 01940, United Sta...","[{'label': 'display', 'lat': 42.53220231344306...",42.532202,-71.035334,01940,MA,58e3485284c4ed36c50b090a
8,The UPS Store,Shipping Store,1268C Broadway,US,Saugus,United States,NaN,7410,"[1268C Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.45513010433513...",42.455130,-71.023057,01906,MA,56991f3b498e4db57b5dbbf9
9,Sherwin-Williams Paint Store,Hardware Store,900 Broadway,US,Saugus,United States,NaN,5860,"[900 Broadway, Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.46923209991908...",42.469232,-71.024433,01906,MA,50623b4fe4b0ef717cd27bf5


In [17]:
dataframe_filtered.name

0                    TUMI Store - Lynnfield
1                  XFINITY Store by Comcast
2                           The Paper Store
3                                AT&T Store
4                            Medicine Store
5                                   C Store
6              Your CBD Store - Reading, MA
7                  XFINITY Store by Comcast
8                             The UPS Store
9              Sherwin-Williams Paint Store
10                             Amazon Books
11                              Ryers Store
12                New Balance Factory Store
13                             Sprint Store
14                             Sprint Store
15               Richdale Convenience Store
16                Andrea's Convenient Store
17                   Jefferys package store
18             Finders Keepers Thrift Store
19                      Richdale Food Store
20                            The UPS Store
21                             Sprint Store
22                  The Computer

In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office

# add a red circle marker to represent the new office
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the stores as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [19]:
#Set the location of the new office
latitude = 42.519
longitude = -71.0433

In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&radius=10000&limit=1000'

In [22]:
import requests
results = requests.get(url).json()
'There are {} around the new office.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around the new office.'

In [23]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '520e153f11d233c87af4422d',
  'name': 'MarketStreet Lynnfield',
  'location': {'address': '600 Market St',
   'crossStreet': 'at Walnut St',
   'lat': 42.51522576983466,
   'lng': -71.03449274955358,
   'labeledLatLngs': [{'label': 'display',
     'lat': 42.51522576983466,
     'lng': -71.03449274955358}],
   'distance': 835,
   'postalCode': '01940',
   'cc': 'US',
   'city': 'Lynnfield',
   'state': 'MA',
   'country': 'United States',
   'formattedAddress': ['600 Market St (at Walnut St)',
    'Lynnfield, MA 01940',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1fd941735',
    'name': 'Shopping Mall',
    'pluralName': 'Shopping Malls',
    'shortName': 'Mall',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 

In [24]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,MarketStreet Lynnfield,Shopping Mall,600 Market St,US,Lynnfield,United States,at Walnut St,835,"[600 Market St (at Walnut St), Lynnfield, MA 0...","[{'label': 'display', 'lat': 42.51522576983466...",42.515226,-71.034493,NaN,01940,MA,520e153f11d233c87af4422d
1,sweetgreen,Salad Place,675 Market St,US,Lynnfield,United States,NaN,676,"[675 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5167886, 'lng'...",42.516789,-71.035619,NaN,01940,MA,56376bfb498e1a42e86b7b73
2,Whole Foods Market,Grocery Store,427 Walnut Street,US,Lynnfield,United States,NaN,1053,"[427 Walnut Street, Lynnfield, MA 01940, Unite...","[{'label': 'display', 'lat': 42.5144224656762,...",42.514422,-71.032063,NaN,01940,MA,521209e111d244c9730c2449
3,Meletharb Homemade Ice Cream,Ice Cream Shop,393 Lowell St,US,Wakefield,United States,NaN,1680,"[393 Lowell St, Wakefield, MA 01880, United St...","[{'label': 'display', 'lat': 42.51736478257608...",42.517365,-71.063662,NaN,01880,MA,4bafeb51f964a520792b3ce3
4,Starbucks,Coffee Shop,420 Market St,US,Lynnfield,United States,NaN,896,"[420 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51516539, 'lng...",42.515165,-71.033691,NaN,01940,MA,5216224911d2c0d3d4626e20
5,California Pizza Kitchen,Pizza Place,"1150 Market Street, Lynnfield",US,Lynnfield,United States,NaN,661,"[1150 Market Street, Lynnfield, Lynnfield, MA ...","[{'label': 'display', 'lat': 42.517421, 'lng':...",42.517421,-71.035522,NaN,01940,MA,5754cef9498e8ab8b029702d
6,Apple MarketStreet,Electronics Store,600 Market St,US,Lynnfield,United States,NaN,817,"[600 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51567799572217...",42.515678,-71.034415,NaN,01940,MA,58637fb5a370b9294234014d
7,J.P. Licks MarketStreet,Ice Cream Shop,800 Market St,US,Lynnfield,United States,NaN,789,"[800 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.51603273747467...",42.516033,-71.034565,NaN,01940,MA,525a0cc711d26b8d7178ef3c
8,Boston Sports Club,Gym / Fitness Center,425 Walnut St,US,Lynnfield,United States,Audubon Rd,738,"[425 Walnut St (Audubon Rd), Lynnfield, MA 019...","[{'label': 'display', 'lat': 42.51476129213743...",42.514761,-71.036369,NaN,01940,MA,4b72b4dff964a520ae842de3
9,Legal C Bar,Seafood Restaurant,220 Market St,US,Lynnfield,United States,NaN,993,"[220 Market St, Lynnfield, MA 01940, United St...","[{'label': 'display', 'lat': 42.5153549476733,...",42.515355,-71.032241,NaN,01940,MA,5250777c8bbdda58c67f00d7


<h4>The family wants to explore the area around the new office building</h4>
The map provides the family with various venues around the new office building.

In [25]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<h4>The family wants to see where all the malls near the new office.</h4>
The map provides the family with the malls near the new office building.

In [26]:
search_query = 'Shopping Mall'
radius = 25000
print(search_query + ' .... OK!')

Shopping Mall .... OK!


In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Shopping Mall&radius=25000&limit=1000'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e4cb51835dd001ba56822'},
 'response': {'venues': [{'id': '472b28e9f964a520f54b1fe3',
    'name': 'Northshore Mall',
    'location': {'address': '210 Andover St',
     'crossStreet': 'at Prospect St',
     'lat': 42.540465,
     'lng': -70.9427116,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.540465,
       'lng': -70.9427116}],
     'distance': 8590,
     'postalCode': '01960',
     'cc': 'US',
     'city': 'Peabody',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['210 Andover St (at Prospect St)',
      'Peabody, MA 01960',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580092723',
    'hasPerk': False},
   {'id': '4563711

In [29]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,472b28e9f964a520f54b1fe3,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,Northshore Mall,v-1580092723,NaN
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,45637114f964a520ab3d1fe3,75 Middlesex Turnpike,US,Burlington,United States,Burlington Mall Rd.,14564,"[75 Middlesex Turnpike (Burlington Mall Rd.), ...","[{'label': 'display', 'lat': 42.4824802, 'lng'...",42.482480,-71.213701,01803,MA,Burlington Mall,v-1580092723,NaN
2,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",False,4b0f0e95f964a520a85e23e3,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,AMC Loews Liberty Tree Mall 20,v-1580092723,NaN
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b4e6158f964a5208ceb26e3,3850 Mystic Valley Pkwy,US,Medford,United States,NaN,13103,"[3850 Mystic Valley Pkwy, Medford, MA 02155, U...","[{'label': 'display', 'lat': 42.40697385154379...",42.406974,-71.092271,02155,MA,Meadow Glen Mall,v-1580092723,57551331
4,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4bbcfabb8ec3d13ab9811b28,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,Woburn Mall,v-1580092723,NaN


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
1,Burlington Mall,Shopping Mall,75 Middlesex Turnpike,US,Burlington,United States,Burlington Mall Rd.,14564,"[75 Middlesex Turnpike (Burlington Mall Rd.), ...","[{'label': 'display', 'lat': 42.4824802, 'lng'...",42.482480,-71.213701,01803,MA,45637114f964a520ab3d1fe3
2,AMC Loews Liberty Tree Mall 20,Movie Theater,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,4b0f0e95f964a520a85e23e3
3,Meadow Glen Mall,Shopping Mall,3850 Mystic Valley Pkwy,US,Medford,United States,NaN,13103,"[3850 Mystic Valley Pkwy, Medford, MA 02155, U...","[{'label': 'display', 'lat': 42.40697385154379...",42.406974,-71.092271,02155,MA,4b4e6158f964a5208ceb26e3
4,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
5,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
6,square one mall-debs,Women's Store,NaN,US,Saugus,United States,NaN,3890,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.48871612548828...",42.488716,-71.019630,01906,MA,50c935cbe4b07110f3609f8e
7,Square One Mall,Shopping Mall,1201 Broadway,US,Saugus,United States,at Essex St,6604,"[1201 Broadway (at Essex St), Saugus, MA 01906...","[{'label': 'display', 'lat': 42.46083411679223...",42.460834,-71.027447,01906,MA,4a56e4b2f964a520e1b51fe3
8,Burtons North Shore Mall,American Restaurant,NaN,US,Danvers,United States,NaN,4439,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5208750208098,...",42.520875,-70.989250,NaN,MA,4dbc51a11e72b351caafa708
9,Watertown Mall,Shopping Mall,550 Arsenal St,US,Watertown,United States,at Elm St.,19753,"[550 Arsenal St (at Elm St.), Watertown, MA 02...","[{'label': 'display', 'lat': 42.36309288530116...",42.363093,-71.158141,02472,MA,4a248752f964a520277e1fe3


In [31]:
#create a list of shopping malls
malls = dataframe_filtered
malls

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
1,Burlington Mall,Shopping Mall,75 Middlesex Turnpike,US,Burlington,United States,Burlington Mall Rd.,14564,"[75 Middlesex Turnpike (Burlington Mall Rd.), ...","[{'label': 'display', 'lat': 42.4824802, 'lng'...",42.482480,-71.213701,01803,MA,45637114f964a520ab3d1fe3
2,AMC Loews Liberty Tree Mall 20,Movie Theater,100 Independence Way,US,Danvers,United States,at Liberty Tree Mall,9193,"[100 Independence Way (at Liberty Tree Mall), ...","[{'label': 'display', 'lat': 42.55266502119669...",42.552665,-70.940950,01923,MA,4b0f0e95f964a520a85e23e3
3,Meadow Glen Mall,Shopping Mall,3850 Mystic Valley Pkwy,US,Medford,United States,NaN,13103,"[3850 Mystic Valley Pkwy, Medford, MA 02155, U...","[{'label': 'display', 'lat': 42.40697385154379...",42.406974,-71.092271,02155,MA,4b4e6158f964a5208ceb26e3
4,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
5,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
6,square one mall-debs,Women's Store,NaN,US,Saugus,United States,NaN,3890,"[Saugus, MA 01906, United States]","[{'label': 'display', 'lat': 42.48871612548828...",42.488716,-71.019630,01906,MA,50c935cbe4b07110f3609f8e
7,Square One Mall,Shopping Mall,1201 Broadway,US,Saugus,United States,at Essex St,6604,"[1201 Broadway (at Essex St), Saugus, MA 01906...","[{'label': 'display', 'lat': 42.46083411679223...",42.460834,-71.027447,01906,MA,4a56e4b2f964a520e1b51fe3
8,Burtons North Shore Mall,American Restaurant,NaN,US,Danvers,United States,NaN,4439,"[Danvers, MA, United States]","[{'label': 'display', 'lat': 42.5208750208098,...",42.520875,-70.989250,NaN,MA,4dbc51a11e72b351caafa708
9,Watertown Mall,Shopping Mall,550 Arsenal St,US,Watertown,United States,at Elm St.,19753,"[550 Arsenal St (at Elm St.), Watertown, MA 02...","[{'label': 'display', 'lat': 42.36309288530116...",42.363093,-71.158141,02472,MA,4a248752f964a520277e1fe3


In [32]:
options = ['Shopping Mall'] 
  
# selecting rows based on condition 
malls_df = malls[dataframe_filtered['categories'].isin(options)] 
  
malls_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Northshore Mall,Shopping Mall,210 Andover St,US,Peabody,United States,at Prospect St,8590,"[210 Andover St (at Prospect St), Peabody, MA ...","[{'label': 'display', 'lat': 42.540465, 'lng':...",42.540465,-70.942712,01960,MA,472b28e9f964a520f54b1fe3
1,Burlington Mall,Shopping Mall,75 Middlesex Turnpike,US,Burlington,United States,Burlington Mall Rd.,14564,"[75 Middlesex Turnpike (Burlington Mall Rd.), ...","[{'label': 'display', 'lat': 42.4824802, 'lng'...",42.482480,-71.213701,01803,MA,45637114f964a520ab3d1fe3
3,Meadow Glen Mall,Shopping Mall,3850 Mystic Valley Pkwy,US,Medford,United States,NaN,13103,"[3850 Mystic Valley Pkwy, Medford, MA 02155, U...","[{'label': 'display', 'lat': 42.40697385154379...",42.406974,-71.092271,02155,MA,4b4e6158f964a5208ceb26e3
4,Woburn Mall,Shopping Mall,296 Mishawum Rd,US,Woburn,United States,NaN,7498,"[296 Mishawum Rd, Woburn, MA 01801, United Sta...","[{'label': 'display', 'lat': 42.50488838716877...",42.504888,-71.132658,01801,MA,4bbcfabb8ec3d13ab9811b28
5,Liberty Tree Mall,Shopping Mall,100 Independence Way,US,Danvers,United States,at Commonwealth Ave,9249,"[100 Independence Way (at Commonwealth Ave), D...","[{'label': 'display', 'lat': 42.5521762, 'lng'...",42.552176,-70.939918,01923,MA,472b2475f964a520f44b1fe3
7,Square One Mall,Shopping Mall,1201 Broadway,US,Saugus,United States,at Essex St,6604,"[1201 Broadway (at Essex St), Saugus, MA 01906...","[{'label': 'display', 'lat': 42.46083411679223...",42.460834,-71.027447,01906,MA,4a56e4b2f964a520e1b51fe3
9,Watertown Mall,Shopping Mall,550 Arsenal St,US,Watertown,United States,at Elm St.,19753,"[550 Arsenal St (at Elm St.), Watertown, MA 02...","[{'label': 'display', 'lat': 42.36309288530116...",42.363093,-71.158141,02472,MA,4a248752f964a520277e1fe3
10,Redstone Shopping Center,Shopping Mall,65 Main St,US,Stoneham,United States,NaN,5449,"[65 Main St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.49509895626455...",42.495099,-71.101248,02180,MA,4c5ff1e5de6920a1a1b79464
11,Middlesex Mall Shopping Center,Shopping Mall,NaN,US,NaN,United States,NaN,14589,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.46994564853443...",42.469946,-71.208128,NaN,Massachusetts,4eab297677c850207f1917ea
13,Mall Play Space,Shopping Mall,NaN,US,NaN,United States,NaN,5996,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.54435729980469...",42.544357,-70.978806,NaN,Massachusetts,4e3d6245b61cb577be75f656


In [34]:
malls_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the new office


# add the new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(malls_map)


# add malls to the map as blue circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(malls_map)

# display map
malls_map

<h4>The family likes to shop at organic grocery stores</h4>
The map provides the family with the locations of organic grocery stores near the new office.

In [35]:
search_query = 'Grocery Store'
radius = 100000
print(search_query + ' .... OK!')

Grocery Store .... OK!


In [36]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Grocery Store&radius=100000&limit=1000'

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e4da647b43d0dca0d344f'},
 'response': {'venues': [{'id': '4e4e52e3bd4101d0d7a7bcfc',
    'name': 'Gourmet Grocery Store',
    'location': {'address': '1868 Massachusetts Ave',
     'lat': 42.387902830824565,
     'lng': -71.11947478486462,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.387902830824565,
       'lng': -71.11947478486462}],
     'distance': 15878,
     'postalCode': '02140',
     'cc': 'US',
     'city': 'Cambridge',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['1868 Massachusetts Ave',
      'Cambridge, MA 02140',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580092818',
    'hasPerk': False

In [38]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4e4e52e3bd4101d0d7a7bcfc,1868 Massachusetts Ave,US,Cambridge,United States,NaN,15878,"[1868 Massachusetts Ave, Cambridge, MA 02140, ...","[{'label': 'display', 'lat': 42.38790283082456...",42.387903,-71.119475,02140,MA,Gourmet Grocery Store,v-1580092818,NaN
1,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4c71a27bd944224bc9be0a29,86 Broadway,US,Somerville,United States,at Franklin St,15043,"[86 Broadway (at Franklin St), Somerville, MA ...","[{'label': 'display', 'lat': 42.38694, 'lng': ...",42.386940,-71.082198,02145,MA,Amigos Grocery Store,v-1580092818,NaN
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,5d5ee4918aa0b70008ff4ab5,334 Lowell St,US,Lawrence,United States,NaN,23269,"[334 Lowell St, Lawrence, MA 01841, United Sta...","[{'label': 'display', 'lat': 42.704875, 'lng':...",42.704875,-71.173254,01841,MA,Jacqueline’s Grocery Store,v-1580092818,NaN
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,51bf85d5e4b0111e5ce1fa10,NaN,US,Andover,United States,NaN,17343,"[Andover, MA, United States]","[{'label': 'display', 'lat': 42.657097, 'lng':...",42.657097,-71.141280,NaN,MA,Smallest Grocery Store In America,v-1580092818,NaN
4,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4c298d16783d9c7488868f6b,303 Columbus Ave Ste 2,US,Boston,United States,NaN,19376,"[303 Columbus Ave Ste 2, Boston, MA 02116, Uni...","[{'label': 'display', 'lat': 42.346464, 'lng':...",42.346464,-71.074500,02116,MA,Discount Grocery Store,v-1580092818,NaN


In [39]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gourmet Grocery Store,Grocery Store,1868 Massachusetts Ave,US,Cambridge,United States,NaN,15878,"[1868 Massachusetts Ave, Cambridge, MA 02140, ...","[{'label': 'display', 'lat': 42.38790283082456...",42.387903,-71.119475,02140,MA,4e4e52e3bd4101d0d7a7bcfc
1,Amigos Grocery Store,Grocery Store,86 Broadway,US,Somerville,United States,at Franklin St,15043,"[86 Broadway (at Franklin St), Somerville, MA ...","[{'label': 'display', 'lat': 42.38694, 'lng': ...",42.386940,-71.082198,02145,MA,4c71a27bd944224bc9be0a29
2,Jacqueline’s Grocery Store,Grocery Store,334 Lowell St,US,Lawrence,United States,NaN,23269,"[334 Lowell St, Lawrence, MA 01841, United Sta...","[{'label': 'display', 'lat': 42.704875, 'lng':...",42.704875,-71.173254,01841,MA,5d5ee4918aa0b70008ff4ab5
3,Smallest Grocery Store In America,Grocery Store,NaN,US,Andover,United States,NaN,17343,"[Andover, MA, United States]","[{'label': 'display', 'lat': 42.657097, 'lng':...",42.657097,-71.141280,NaN,MA,51bf85d5e4b0111e5ce1fa10
4,Discount Grocery Store,Department Store,303 Columbus Ave Ste 2,US,Boston,United States,NaN,19376,"[303 Columbus Ave Ste 2, Boston, MA 02116, Uni...","[{'label': 'display', 'lat': 42.346464, 'lng':...",42.346464,-71.074500,02116,MA,4c298d16783d9c7488868f6b
5,Shin Shin Oriental Grocery Store,Grocery Store,615 S Union St,US,Lawrence,United States,NaN,20712,"[615 S Union St, Lawrence, MA, United States]","[{'label': 'display', 'lat': 42.68651405088592...",42.686514,-71.153320,NaN,MA,4b7c8dd0f964a520b39a2fe3
6,Bob's Grocery Store,Convenience Store,511-535 Andover St,US,Lawrence,United States,NaN,21730,"[511-535 Andover St, Lawrence, MA 01843, Unite...","[{'label': 'display', 'lat': 42.691386, 'lng':...",42.691386,-71.167742,01843,MA,4e137901483bee47ff43ecee
7,Gigi's Liquor and Grocery Store,Liquor Store,123 E Main St,US,Merrimac,United States,NaN,35766,"[123 E Main St, Merrimac, MA 01860, United Sta...","[{'label': 'display', 'lat': 42.836904, 'lng':...",42.836904,-70.979935,01860,MA,4f326b5519836c91c7d5fae5
8,TUMI Store - Lynnfield,Luggage Store,600 Market St Spc 687,US,Lynnfield,United States,NaN,707,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,5be1033e92e7a9002c62ce84
9,XFINITY Store by Comcast,Business Service,601 Edgewater Dr,US,Wakefield,United States,NaN,613,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,57e39d8f498ee24a37e658dc


In [40]:
#create a list of grocery stores
grocery = dataframe_filtered
grocery

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gourmet Grocery Store,Grocery Store,1868 Massachusetts Ave,US,Cambridge,United States,NaN,15878,"[1868 Massachusetts Ave, Cambridge, MA 02140, ...","[{'label': 'display', 'lat': 42.38790283082456...",42.387903,-71.119475,02140,MA,4e4e52e3bd4101d0d7a7bcfc
1,Amigos Grocery Store,Grocery Store,86 Broadway,US,Somerville,United States,at Franklin St,15043,"[86 Broadway (at Franklin St), Somerville, MA ...","[{'label': 'display', 'lat': 42.38694, 'lng': ...",42.386940,-71.082198,02145,MA,4c71a27bd944224bc9be0a29
2,Jacqueline’s Grocery Store,Grocery Store,334 Lowell St,US,Lawrence,United States,NaN,23269,"[334 Lowell St, Lawrence, MA 01841, United Sta...","[{'label': 'display', 'lat': 42.704875, 'lng':...",42.704875,-71.173254,01841,MA,5d5ee4918aa0b70008ff4ab5
3,Smallest Grocery Store In America,Grocery Store,NaN,US,Andover,United States,NaN,17343,"[Andover, MA, United States]","[{'label': 'display', 'lat': 42.657097, 'lng':...",42.657097,-71.141280,NaN,MA,51bf85d5e4b0111e5ce1fa10
4,Discount Grocery Store,Department Store,303 Columbus Ave Ste 2,US,Boston,United States,NaN,19376,"[303 Columbus Ave Ste 2, Boston, MA 02116, Uni...","[{'label': 'display', 'lat': 42.346464, 'lng':...",42.346464,-71.074500,02116,MA,4c298d16783d9c7488868f6b
5,Shin Shin Oriental Grocery Store,Grocery Store,615 S Union St,US,Lawrence,United States,NaN,20712,"[615 S Union St, Lawrence, MA, United States]","[{'label': 'display', 'lat': 42.68651405088592...",42.686514,-71.153320,NaN,MA,4b7c8dd0f964a520b39a2fe3
6,Bob's Grocery Store,Convenience Store,511-535 Andover St,US,Lawrence,United States,NaN,21730,"[511-535 Andover St, Lawrence, MA 01843, Unite...","[{'label': 'display', 'lat': 42.691386, 'lng':...",42.691386,-71.167742,01843,MA,4e137901483bee47ff43ecee
7,Gigi's Liquor and Grocery Store,Liquor Store,123 E Main St,US,Merrimac,United States,NaN,35766,"[123 E Main St, Merrimac, MA 01860, United Sta...","[{'label': 'display', 'lat': 42.836904, 'lng':...",42.836904,-70.979935,01860,MA,4f326b5519836c91c7d5fae5
8,TUMI Store - Lynnfield,Luggage Store,600 Market St Spc 687,US,Lynnfield,United States,NaN,707,"[600 Market St Spc 687, Lynnfield, MA 01940, U...","[{'label': 'display', 'lat': 42.5164879, 'lng'...",42.516488,-71.035382,01940,MA,5be1033e92e7a9002c62ce84
9,XFINITY Store by Comcast,Business Service,601 Edgewater Dr,US,Wakefield,United States,NaN,613,"[601 Edgewater Dr, Wakefield, MA 01880, United...","[{'label': 'display', 'lat': 42.52202165068454...",42.522022,-71.049560,01880,MA,57e39d8f498ee24a37e658dc


In [41]:
options = ['Grocery Store'] 
  
# selecting rows based on condition 
grocery_df = grocery[dataframe_filtered['categories'].isin(options)] 
  
grocery_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gourmet Grocery Store,Grocery Store,1868 Massachusetts Ave,US,Cambridge,United States,NaN,15878,"[1868 Massachusetts Ave, Cambridge, MA 02140, ...","[{'label': 'display', 'lat': 42.38790283082456...",42.387903,-71.119475,02140,MA,4e4e52e3bd4101d0d7a7bcfc
1,Amigos Grocery Store,Grocery Store,86 Broadway,US,Somerville,United States,at Franklin St,15043,"[86 Broadway (at Franklin St), Somerville, MA ...","[{'label': 'display', 'lat': 42.38694, 'lng': ...",42.386940,-71.082198,02145,MA,4c71a27bd944224bc9be0a29
2,Jacqueline’s Grocery Store,Grocery Store,334 Lowell St,US,Lawrence,United States,NaN,23269,"[334 Lowell St, Lawrence, MA 01841, United Sta...","[{'label': 'display', 'lat': 42.704875, 'lng':...",42.704875,-71.173254,01841,MA,5d5ee4918aa0b70008ff4ab5
3,Smallest Grocery Store In America,Grocery Store,NaN,US,Andover,United States,NaN,17343,"[Andover, MA, United States]","[{'label': 'display', 'lat': 42.657097, 'lng':...",42.657097,-71.141280,NaN,MA,51bf85d5e4b0111e5ce1fa10
5,Shin Shin Oriental Grocery Store,Grocery Store,615 S Union St,US,Lawrence,United States,NaN,20712,"[615 S Union St, Lawrence, MA, United States]","[{'label': 'display', 'lat': 42.68651405088592...",42.686514,-71.153320,NaN,MA,4b7c8dd0f964a520b39a2fe3
31,Xpect Grocery & Deep Discount Stores,Grocery Store,436 Broadway,US,Methuen,United States,NaN,27035,"[436 Broadway, Methuen, MA 01844, United States]","[{'label': 'display', 'lat': 42.735078, 'lng':...",42.735078,-71.193990,01844,MA,5239d48911d2da4becb86914
40,Walmart Grocery Pickup & Delivery,Grocery Store,326 N Broadway,US,Salem,United States,NaN,35413,"[326 N Broadway, Salem, NH 03079, United States]","[{'label': 'display', 'lat': 42.8016584011, 'l...",42.801658,-71.241800,03079,NH,5c4b7ee4f129b5002555683d
45,Walmart Grocery Pickup & Delivery,Grocery Store,550 Providence Hwy,US,Walpole,United States,NaN,43645,"[550 Providence Hwy, Walpole, MA 02081, United...","[{'label': 'display', 'lat': 42.1480617476224,...",42.148062,-71.215112,02081,MA,5c4b807fb3c961002c3c1524
46,Walmart Grocery Pickup & Delivery,Grocery Store,30 Memorial Dr,US,Avon,United States,NaN,44265,"[30 Memorial Dr, Avon, MA 02322, United States]","[{'label': 'display', 'lat': 42.12143, 'lng': ...",42.121430,-71.032940,02322,MA,5aab07aeee71203bce3f0806


In [43]:
grocery_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around Boston's Old City Hall


# add Bostons Old City Hall as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Old City Hall',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(grocery_map)


# add grocery to the map as blue circle markers
for lat, lng, label in zip(grocery_df.lat, grocery_df.lng, grocery_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(grocery_map)

# display map
grocery_map

<h4>The family likes to go to a gym three times per week.</h4>
The map provides the family with the locations of gyms near the new office.

In [56]:
search_query = 'Gym'
radius = 10000
print(search_query + ' .... OK!')

Gym .... OK!


In [57]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Gym&radius=10000&limit=1000'

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e4efd6d8c56001bb5f0df'},
 'response': {'venues': [{'id': '4e2a06afd4c0e5c89ab4ea4e',
    'name': 'Gym',
    'location': {'address': 'Best Western New Englander',
     'crossStreet': '1 Rainin Road',
     'lat': 42.48803964960014,
     'lng': -71.11794289833213,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.48803964960014,
       'lng': -71.11794289833213}],
     'distance': 7028,
     'postalCode': '01801',
     'cc': 'US',
     'city': 'Woburn',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Best Western New Englander (1 Rainin Road)',
      'Woburn, MA 01801',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580093213',
    'hasPerk': False},

In [59]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4e2a06afd4c0e5c89ab4ea4e,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,Gym,v-1580093213
1,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",False,57166a12498e33ed2245020a,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,Gold's Gym,v-1580093213
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4d436d122e326ea8778dd9a6,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,Gym at Edgewood,v-1580093213
3,"[{'id': '4bf58dd8d48988d1a9941735', 'name': 'C...",False,4ba4ff38f964a5201ece38e3,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,The Little Gym of Danvers,v-1580093213
4,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,5aa9217d92e7a94231c2e03b,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,fuji gym,v-1580093213


In [60]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
1,Gold's Gym,Athletics & Sports,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,57166a12498e33ed2245020a
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
3,The Little Gym of Danvers,College Rec Center,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,4ba4ff38f964a5201ece38e3
4,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
5,gym source at Jordan's Furniture,Sporting Goods Shop,NaN,US,Reading,United States,NaN,3911,"[Reading, MA, United States]","[{'label': 'display', 'lat': 42.520514, 'lng':...",42.520514,-71.090935,NaN,MA,574b615b498eeaadc7fb23a7
6,gym,Track,NaN,US,NaN,United States,NaN,7785,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.57612187469009...",42.576122,-70.988524,NaN,Massachusetts,4f55a5c3e4b072b51d782d35
7,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
8,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
9,Gymboree,Kids Store,NaN,US,Lynnfield,United States,NaN,902,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.515327, 'lng':...",42.515327,-71.033501,NaN,MA,523e154211d26f23616fe504


In [61]:
#create a list of gyms
gym = dataframe_filtered
gym

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
1,Gold's Gym,Athletics & Sports,1 General Way,US,Reading,United States,NaN,4702,"[1 General Way, Reading, MA 01876, United States]","[{'label': 'display', 'lat': 42.51933951918963...",42.519340,-71.100610,01876,MA,57166a12498e33ed2245020a
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
3,The Little Gym of Danvers,College Rec Center,29 Andover St,US,Danvers,United States,NaN,7958,"[29 Andover St, Danvers, MA 01923, United States]","[{'label': 'display', 'lat': 42.55246781136085...",42.552468,-70.957562,01923,MA,4ba4ff38f964a5201ece38e3
4,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
5,gym source at Jordan's Furniture,Sporting Goods Shop,NaN,US,Reading,United States,NaN,3911,"[Reading, MA, United States]","[{'label': 'display', 'lat': 42.520514, 'lng':...",42.520514,-71.090935,NaN,MA,574b615b498eeaadc7fb23a7
6,gym,Track,NaN,US,NaN,United States,NaN,7785,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.57612187469009...",42.576122,-70.988524,NaN,Massachusetts,4f55a5c3e4b072b51d782d35
7,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
8,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
9,Gymboree,Kids Store,NaN,US,Lynnfield,United States,NaN,902,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.515327, 'lng':...",42.515327,-71.033501,NaN,MA,523e154211d26f23616fe504


In [62]:
options = ['Gym', 'Gym / Fitness Center'] 
  
# selecting rows based on condition 
gym_df = gym[dataframe_filtered['categories'].isin(options)] 
  
gym_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym,Best Western New Englander,US,Woburn,United States,1 Rainin Road,7028,"[Best Western New Englander (1 Rainin Road), W...","[{'label': 'display', 'lat': 42.48803964960014...",42.488040,-71.117943,01801,MA,4e2a06afd4c0e5c89ab4ea4e
2,Gym at Edgewood,Gym,100 Lowell Rd,US,North Reading,United States,NaN,9789,"[100 Lowell Rd, North Reading, MA 01864, Unite...","[{'label': 'display', 'lat': 42.58407120701979...",42.584071,-71.123601,01864,MA,4d436d122e326ea8778dd9a6
4,fuji gym,Gym / Fitness Center,19 New Salem St,US,Wakefield,United States,NaN,2336,"[19 New Salem St, Wakefield, MA 01880, United ...","[{'label': 'display', 'lat': 42.505176, 'lng':...",42.505176,-71.064734,01880,MA,5aa9217d92e7a94231c2e03b
7,Public Safety Bldg Gym,Gym / Fitness Center,1 Union St,US,Wakefield,United States,princess st,2697,"[1 Union St (princess st), Wakefield, MA 01880...","[{'label': 'display', 'lat': 42.504199, 'lng':...",42.504199,-71.069333,01880,MA,4db98d81fa8c978590d28226
8,Mellow Yellow Gym,Gym,NaN,US,Lynnfield,United States,NaN,2812,"[Lynnfield, MA, United States]","[{'label': 'display', 'lat': 42.53879419905821...",42.538794,-71.064613,NaN,MA,57080ac6cd100e4615608394
10,Quietman Sports Gym,Gym,NaN,US,Lynnfield,United States,NaN,3311,"[Lynnfield, MA 01940, United States]","[{'label': 'display', 'lat': 42.510996, 'lng':...",42.510996,-71.004426,01940,MA,4f914c03e4b0ff5a5163b3b3
12,Mike's Gym,Gym / Fitness Center,10 Broadway St,US,Wakefield,United States,NaN,3291,"[10 Broadway St, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.49717334246895...",42.497173,-71.070364,01880,MA,4e5e3f531838f725527300ab
13,Gym,Gym,NaN,US,NaN,United States,NaN,11846,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.4212517753186,...",42.421252,-70.986272,NaN,Massachusetts,4e89f2b846909b71217e6407
15,300 Unicorn Park Gym,Gym,NaN,US,Woburn,United States,NaN,7096,"[Woburn, MA 01801, United States]","[{'label': 'display', 'lat': 42.48357742432995...",42.483577,-71.115190,01801,MA,4f5125bbe4b0442190af0dc5
16,Powerfit Gym,Gym,NaN,US,Peabody,United States,NaN,5814,"[Peabody, MA 01960, United States]","[{'label': 'display', 'lat': 42.546903, 'lng':...",42.546903,-70.983381,01960,MA,50f16751e4b036c5c6ad8742


In [65]:
gym_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(gym_map)


# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(gym_map)

# display map
gym_map

In [67]:
search_query = 'Train'
radius = 10000
print(search_query + ' .... OK!')

Train .... OK!


In [68]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PZYWQETWL5NRJQBZAFY0FZJ2ZPEAKBAWQ2WFQAO3VT3WCPEH&client_secret=RSQ1F3JKGNHF0K3UIEEIKWY4T3UXQSAPQ2YDEB3VCM51KHCE&ll=42.519,-71.0433&v=20180604&query=Train&radius=10000&limit=1000'

In [69]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e51070cc1fd001b2de3fd'},
 'response': {'venues': [{'id': '5832d1d0dfa6ff0f03038b5a',
    'name': 'MBTA Commuter Rail Train #200',
    'location': {'lat': 42.521294,
     'lng': -71.10756,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.521294,
       'lng': -71.10756}],
     'distance': 5278,
     'postalCode': '01867',
     'cc': 'US',
     'city': 'Reading',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Reading, MA 01867', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd931735',
      'name': 'Metro Station',
      'pluralName': 'Metro Stations',
      'shortName': 'Metro',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1580093754',
    'hasPerk': False},
   {'id': '50226d7be4b079960ffad394',
    'name': 'MBTA Commuter Rail Train #214',
    'location': {'address': '12:41pm Hav

In [70]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,5832d1d0dfa6ff0f03038b5a,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,MBTA Commuter Rail Train #200,v-1580093754,NaN
1,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,50226d7be4b079960ffad394,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,MBTA Commuter Rail Train #214,v-1580093754,NaN
2,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,4fc4b26ee4b02051856d9e23,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,MBTA Commuter Rail Train #260,v-1580093754,NaN
3,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,5b7a9b1d531593002c415517,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,Mbta Commuter Rail Train #204,v-1580093754,NaN
4,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,5343dce7498e262e20cea692,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,MBTA Commuter Rail Train #220,v-1580093754,NaN


In [71]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
7,"American Training, Inc.",Community Center,107 Audubon Rd,US,Wakefield,United States,NaN,535,"[107 Audubon Rd, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.520594, 'lng':...",42.520594,-71.049453,01880,MA,57a87f42498ea46a2e6547bb
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
9,Stone Zoo Cafe/Train Ride,Zoo Exhibit,149 Pond St,US,Stoneham,United States,NaN,7449,"[149 Pond St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.46297, 'lng': ...",42.462970,-71.092930,02180,MA,5aca5c476bdee60dc841ef91


In [72]:
#create a list of train stations
train = dataframe_filtered
train

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
7,"American Training, Inc.",Community Center,107 Audubon Rd,US,Wakefield,United States,NaN,535,"[107 Audubon Rd, Wakefield, MA 01880, United S...","[{'label': 'display', 'lat': 42.520594, 'lng':...",42.520594,-71.049453,01880,MA,57a87f42498ea46a2e6547bb
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
9,Stone Zoo Cafe/Train Ride,Zoo Exhibit,149 Pond St,US,Stoneham,United States,NaN,7449,"[149 Pond St, Stoneham, MA 02180, United States]","[{'label': 'display', 'lat': 42.46297, 'lng': ...",42.462970,-71.092930,02180,MA,5aca5c476bdee60dc841ef91


In [73]:
options = ['Metro Station', 'Train'] 
  
# selecting rows based on condition 
train_df = train[dataframe_filtered['categories'].isin(options)] 
  
train_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,MBTA Commuter Rail Train #200,Metro Station,NaN,US,Reading,United States,NaN,5278,"[Reading, MA 01867, United States]","[{'label': 'display', 'lat': 42.521294, 'lng':...",42.521294,-71.107560,01867,MA,5832d1d0dfa6ff0f03038b5a
1,MBTA Commuter Rail Train #214,Train,12:41pm Haverhill to Boston,US,NaN,United States,NaN,2051,"[12:41pm Haverhill to Boston, MA 02148, United...","[{'label': 'display', 'lat': 42.52117593798743...",42.521176,-71.068130,02148,Massachusetts,50226d7be4b079960ffad394
2,MBTA Commuter Rail Train #260,Train,NaN,US,NaN,United States,7:30AM Inbound,5278,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.52199837297112...",42.521998,-71.107509,NaN,Massachusetts,4fc4b26ee4b02051856d9e23
3,Mbta Commuter Rail Train #204,Train,Haverhill to North Station,US,Reading,United States,NaN,5248,"[Haverhill to North Station, Reading, MA 01867...","[{'label': 'display', 'lat': 42.52195, 'lng': ...",42.521950,-71.107140,01867,MA,5b7a9b1d531593002c415517
4,MBTA Commuter Rail Train #220,Train,12PM Haverhill to North Station,US,Wakefield,United States,NaN,3554,"[12PM Haverhill to North Station, Wakefield, M...","[{'label': 'display', 'lat': 42.51395034790039...",42.513950,-71.086075,NaN,MA,5343dce7498e262e20cea692
5,MBTA Commuter Rail Train #2204,Train,Haverhill To Boston,US,Wakefield,United States,NaN,3677,"[Haverhill To Boston, Wakefield, MA, United St...","[{'label': 'display', 'lat': 42.49743270874023...",42.497433,-71.077248,NaN,MA,53662ff1498e5b725e6ab340
6,MBTA Commuter Rail Train #321,Train,North Station,US,Boston,United States,NaN,3672,"[North Station, Boston, MA, United States]","[{'label': 'display', 'lat': 42.49840927124023...",42.498409,-71.078270,NaN,MA,5232f98c11d2bbaf28c2445e
8,Train To Boston,Train,NaN,US,Wilmington,United States,NaN,7783,"[Wilmington, MA, United States]","[{'label': 'display', 'lat': 42.51133368679578...",42.511334,-71.137584,NaN,MA,4e2c5c04fa76bbf847e2d324
11,Train To Salem,Train,NaN,US,NaN,United States,NaN,9721,"[Massachusetts, United States]","[{'label': 'display', 'lat': 42.45290592062626...",42.452906,-70.965892,NaN,Massachusetts,4eaad65bcc217eb08e7f0219
12,Mbta Commuter Rail Train #2208,Train,7:50pm Reading to North Station,US,Reading,United States,NaN,5301,"[7:50pm Reading to North Station, Reading, MA ...","[{'label': 'display', 'lat': 42.522045, 'lng':...",42.522045,-71.107790,01867,MA,5dbf73776a99d10008b7ee16


In [74]:
train_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centered around the new office


# add new office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(train_map)


# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(train_map)

# display map
train_map

In [75]:
combined_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centered around the new office


# add New Office as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='New Office',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(combined_map)

# add malls to the map as green circle markers
for lat, lng, label in zip(malls_df.lat, malls_df.lng, malls.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(combined_map)


# add grocery to the map as blue circle markers
for lat, lng, label in zip(grocery_df.lat, grocery_df.lng, grocery_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(combined_map)
    
# add gyms to the map as purple circle markers
for lat, lng, label in zip(gym_df.lat, gym_df.lng, gym_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='purple',
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(combined_map)
    
# add train stations to the map as green circle markers
for lat, lng, label in zip(train_df.lat, train_df.lng, train_df.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
        ).add_to(combined_map)

# display map
combined_map

<h4>The family wants to see where all the malls are located throughout greater Boston</h4>
The map provides the family with the malls around greater Boston.